In [1]:
import pandas as pd
from pandas_plink import read_plink1_bin, write_plink1_bin
import numpy as np
import anndata
import os

In [2]:
adni_1_path = 'ADNI_cluster_01_forward_757LONI'
adni_2_1_path = 'ADNI_GO_2_Forward_Bin'
adni_2_2_path = 'ADNI_GO2_GWAS_2nd_orig_BIN'
adni_3_1_path = 'ADNI3_PLINK_Final'
adni_3_2_path = 'ADNI3_PLINK_FINAL_2nd'

# 1. Prepare liftover format

In [3]:
adni_paths = [adni_1_path, adni_2_1_path, adni_2_2_path, adni_3_1_path, adni_3_2_path] 

for i in range(5):
    df = pd.read_csv('./' + adni_paths[i] + '.bim', header=None, sep='\t')
    
    valid_bases = ['A', 'C', 'G', 'T']
    df_filtered = df[df[4].isin(valid_bases) & df[5].isin(valid_bases)]
    df_filtered = df_filtered.reset_index(drop=True)
    df_filtered.to_csv('./' + adni_paths[i] + '.bim', index=False, header=None, sep='\t')

In [4]:
adni_paths = [adni_1_path, adni_2_1_path, adni_2_2_path, adni_3_1_path, adni_3_2_path]
save_paths = ['adni_1.bed', 'adni_2_1.bed', 'adni_2_2.bed', 'adni_3_1.bed', 'adni_3_2.bed']
os.makedirs('./before_liftover', exist_ok=True)

for i in range(len(adni_paths)):
    bim_path = adni_paths[i]
    save_path = './before_liftover/' + save_paths[i]
    tmp = pd.read_csv(f'./{bim_path}.bim', header=None, sep='\t')
    
    chr_id = tmp[0].replace({23: 'X', 24: 'Y'}).where((0 < tmp[0]) & (tmp[0] < 25), None)
    
    filtered_data = tmp[chr_id.notna()]
    
    new_tmp = pd.DataFrame({
        '0': 'chr' + chr_id[chr_id.notna()].astype(str),
        '1': filtered_data[3],
        '2': filtered_data[3],
        '3': filtered_data[1]
    })
    
    new_tmp = new_tmp.reset_index(drop=True)
    new_tmp.to_csv(save_path, index=False, sep='\t', header=None)

# 2. Run Liftover & Filtering

In [5]:
### --------------- Liftover Process (Run following commandas at the terminal) --------------- ###
# wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/liftOver
# chmod +x liftOver
# wget http://hgdownload.soe.ucsc.edu/goldenPath/hg18/liftOver/hg18ToHg38.over.chain.gz
# ./liftOver ./before_liftover/adni_1.bed hg18ToHg38.over.chain.gz ./after_liftover/ADNI_1_Hg38.bed ./after_liftover/unlifted_ADNI_1_Hg38.bed 
# ./liftOver ./before_liftover/adni_2_1.bed hg18ToHg38.over.chain.gz ./after_liftover/ADNI_2_1_Hg38.bed ./after_liftover/unlifted_ADNI_2_1_Hg38.bed 
# ./liftOver ./before_liftover/adni_2_2.bed hg18ToHg38.over.chain.gz ./after_liftover/ADNI_2_2_Hg38.bed ./after_liftover/unlifted_ADNI_2_2_Hg38.bed 
# ./liftOver ./before_liftover/adni_3_1.bed hg18ToHg38.over.chain.gz ./after_liftover/ADNI_3_1_Hg38.bed ./after_liftover/unlifted_ADNI_3_1_Hg38.bed 
# ./liftOver ./before_liftover/adni_3_2.bed hg18ToHg38.over.chain.gz ./after_liftover/ADNI_3_2_Hg38.bed ./after_liftover/unlifted_ADNI_3_2_Hg38.bed 

In [6]:
# filtering
os.makedirs('./after_liftover', exist_ok=True)
after_liftover_paths = ['after_liftover/ADNI_1', 'after_liftover/ADNI_2_1', 'after_liftover/ADNI_2_2', 'after_liftover/ADNI_3_1', 'after_liftover/ADNI_3_2']

for after_liftover_path in after_liftover_paths:
    _tmp = pd.read_csv(f'./' + after_liftover_path + '_Hg38.bed', header=None, sep='\t')
    _tmp_filtered = _tmp[~_tmp[0].str.contains('_')]
    _tmp_filtered.to_csv(f'./' + after_liftover_path + '_Hg38.bed', index=False, header=False, sep='\t')

# 3. Align .bim, .bed, .fam with liftover ids

In [7]:
adni_paths = [adni_1_path, adni_2_1_path, adni_2_2_path, adni_3_1_path, adni_3_2_path]
liftover_paths = ['after_liftover/ADNI_1', 'after_liftover/ADNI_2_1', 'after_liftover/ADNI_2_2', 'after_liftover/ADNI_3_1', 'after_liftover/ADNI_3_2']
output_paths = ['liftovered/ADNI_1', 'liftovered/ADNI_2_1', 'liftovered/ADNI_2_2', 'liftovered/ADNI_3_1', 'liftovered/ADNI_3_2']
os.makedirs('./liftovered', exist_ok=True)

for i in range(5):
    adni_path = './' + adni_paths[i]
    liftover_path = './' + liftover_paths[i] + '_Hg38.bed'
    output_path = './' + output_paths[i]
    
    G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')

    # Read the liftover file to get the new SNP IDs
    liftover = pd.read_csv(liftover_path, sep='\t', header=None, names=['chr', 'pos1', 'pos2', 'id'])

    # Find indices of the SNPs in bim that are present in the liftover file
    indices = np.arange(G.shape[1])[(pd.DataFrame(G.snp)[0].isin(liftover['id']))]

    if i == 0:
        fam = pd.read_csv('./ADNI_cluster_01_forward_757LONI.fam', sep=' ', header=None, names=['fid', 'iid', 'father', 'mother', 'gender', 'trait'])
        fam_indices = ~fam['iid'].isin(['073_S_0909', '130_S_1201'])
        _G = G[fam_indices, indices]

    else:
        _G = G[:, indices]
    
    _G['chrom'] = ('variant', np.array(liftover['chr']))
    _G['pos'] = ('variant', np.array(liftover['pos1']))
    write_plink1_bin(_G, output_path+'.bed', output_path+'.bim', output_path+'.fam')

Mapping files:   0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_3706173/1774242314.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Mapping files:  33%|███▎      | 1/3 [00:00<00:00,  2.32it/s]/tmp/ipykernel_3706173/1774242314.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Writing BED: 100%|██████████| 2/2 [00:08<00:00,  4.08s/it]

Writing FAM... done.
Writing BIM... 

done.


Mapping files:   0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_3706173/1774242314.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Mapping files:  33%|███▎      | 1/3 [00:00<00:00,  2.18it/s]/tmp/ipykernel_3706173/1774242314.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Writing BED: 100%|██████████| 1/1 [00:03<00:00,  3.87s/it]

Writing FAM... done.
Writing BIM... 

...

Writing FAM... done.
Writing BIM... 

done.


Mapping files:   0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_3706173/1774242314.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Mapping files:  33%|███▎      | 1/3 [00:00<00:00,  2.33it/s]/tmp/ipykernel_3706173/1774242314.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Writing BED: 100%|██████████| 1/1 [00:03<00:00,  3.21s/it]

Writing FAM... done.
Writing BIM... 

done.


In [8]:
G_list = []
for i in range(5):
    adni_path = './' + output_paths[i]
    
    G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
    G_list.append(G)

Mapping files:   0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_3706173/1243287097.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Mapping files:  33%|███▎      | 1/3 [00:00<00:00,  2.28it/s]/tmp/ipykernel_3706173/1243287097.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Mapping files:   0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_3706173/1243287097.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G = read_plink1_bin(adni_path+'.bed', adni_path+'.bim', adni_path+'.fam')
Mapping files:  33%|███▎      | 1/3 [00:00<00:00,  2.38it/s]/t

# 4.Merge using plink on your local enviroment

In [9]:
### --------------- Merge using plink (Run following commandas at the terminal) --------------- ###
# wget https://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20241022.zip
# unzip plink_linux_x86_64_20241022.zip -d plink_install
# cd plink_install
# cp plink /usr/local/bin
# sudo cp plink /usr/local/bin
# sudo chmod 755 /usr/local/bin/plink
# sudo nano ~/.bashrc -> export PATH=/usr/local/bin:$PATH
# cd ..
# echo "./liftovered/ADNI_2_1" > ./liftovered/all_datasets.txt
# echo "./liftovered/ADNI_2_2" >> ./liftovered/all_datasets.txt
# echo "./liftovered/ADNI_3_1" >> ./liftovered/all_datasets.txt
# echo "./liftovered/ADNI_3_2" >> ./liftovered/all_datasets.txt
# plink --bfile ./liftovered/ADNI_1 --merge-list ./liftovered/all_datasets.txt --make-bed --out ./liftovered/ADNI_merged

In [10]:
### --------------- LD pruning using plink (Run following commandas at the terminal) --------------- ###
# plink --bfile ./liftovered/ADNI_merged --indep-pairwise 50 5 0.1 --out ./liftovered/ADNI_merged_pruned
# plink --bfile ./liftovered/ADNI_merged --extract ./liftovered/ADNI_merged_pruned.prune.in --make-bed --out ./liftovered/ADNI_final

In [11]:
merged_path = "./liftovered/ADNI_final"
G_merge = read_plink1_bin(merged_path+'.bed', merged_path+'.bim', merged_path+'.fam')

genotype_df = G_merge.to_pandas()
genotype_df.columns = G_merge.snp

adata = anndata.AnnData(X=genotype_df.values,
                        obs=pd.DataFrame(index=genotype_df.index),
                        var=pd.DataFrame(index=genotype_df.columns))

adata.write_h5ad("genomic_merged.h5ad")

Mapping files:   0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_3706173/1561454553.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G_merge = read_plink1_bin(merged_path+'.bed', merged_path+'.bim', merged_path+'.fam')
Mapping files:  33%|███▎      | 1/3 [00:00<00:00,  8.38it/s]/tmp/ipykernel_3706173/1561454553.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  G_merge = read_plink1_bin(merged_path+'.bed', merged_path+'.bim', merged_path+'.fam')
Mapping files: 100%|██████████| 3/3 [00:00<00:00, 11.77it/s]
